# LangChain Expression Language (LCEL)

El Lenguaje de Expresión de Cadenas de LangChain, o LCEL, es una manera declarativa de componer fácilmente cadenas juntas. El LCEL fue diseñado desde el primer día para admitir la implementación de prototipos en producción, sin cambios de código, desde la cadena más simple de "prompt + LLM" hasta las cadenas más complejas (hemos visto a personas ejecutar con éxito cadenas LCEL con 100s de pasos en producción). Para resaltar algunas de las razones por las que podrías querer usar LCEL:

Soporte de transmisión: al construir tus cadenas con LCEL, obtienes el mejor tiempo posible hasta el primer token (tiempo transcurrido hasta que sale el primer fragmento de salida). Para algunas cadenas, esto significa, por ejemplo, transmitir tokens directamente desde un LLM a un analizador de salida en continuo, y obtener fragmentos analizados e incrementales de salida a la misma velocidad que el proveedor de LLM emite los tokens en bruto.

Soporte asíncrono: cualquier cadena construida con LCEL puede ser llamada tanto con la API síncrona (por ejemplo, en tu cuaderno Jupyter durante la prototipificación) como con la API asíncrona (por ejemplo, en un servidor LangServe). Esto permite utilizar el mismo código para prototipos y en producción, con un rendimiento excelente y la capacidad de manejar muchas solicitudes concurrentes en el mismo servidor.

Ejecución paralela optimizada: siempre que las cadenas LCEL tengan pasos que se puedan ejecutar en paralelo (por ejemplo, si recuperas documentos de varios recuperadores), lo hacemos automáticamente, tanto en las interfaces síncronas como asíncronas, para la latencia más pequeña posible.

Reintentos y alternativas: configura reintentos y alternativas para cualquier parte de tu cadena LCEL. Esta es una excelente manera de hacer que tus cadenas sean más confiables a escala. Actualmente estamos trabajando en agregar soporte de transmisión para reintentos/alternativas, para que puedas obtener confiabilidad adicional sin costo de latencia.

Acceso a resultados intermedios: para cadenas más complejas, a menudo es muy útil acceder a los resultados de pasos intermedios incluso antes de que se produzca la salida final. Esto se puede usar para informar a los usuarios finales que algo está sucediendo, o incluso solo para depurar tu cadena. Puedes transmitir resultados intermedios, y está disponible en cada servidor LangServe.

Esquemas de entrada y salida: los esquemas de entrada y salida proporcionan a cada cadena LCEL esquemas Pydantic y JSONSchema inferidos a partir de la estructura de tu cadena. Esto se puede usar para la validación de entradas y salidas, y es una parte integral de LangServe.

Integración transparente con el seguimiento de LangSmith: a medida que tus cadenas se vuelven más y más complejas, es cada vez más importante entender qué está sucediendo exactamente en cada paso. Con LCEL, todos los pasos se registran automáticamente en LangSmith para una máxima observabilidad y capacidad de depuración.

Integración transparente con el despliegue de LangServe: cualquier cadena creada con LCEL se puede implementar fácilmente utilizando LangServe.

# Comenzar
LCEL facilita la construcción de cadenas complejas a partir de componentes básicos y admite funcionalidades listas para usar, como transmisión, paralelismo y registro.

## Ejemplo básico: prompt + modelo + analizador de salida
El caso de uso más básico y común es encadenar una plantilla de prompt y un modelo. Para ver cómo funciona esto, creemos una cadena que tome un tema y genere un chiste:


In [1]:
%pip install --upgrade --quiet langchain-core langchain-community langchain-openai

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Cuentame un chiste {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "procrastinación"})

'¿Sabes cuál es el chiste favorito de los procrastinadores?\n\nMañana te lo cuento.'

Observa esta línea de código, donde ensamblamos diferentes componentes en una única cadena utilizando LCEL:

In [5]:
chain = prompt | model | output_parser

El símbolo | es similar a un operador de tubería de Unix, que encadena los diferentes componentes alimentando la salida de un componente como entrada al siguiente componente.

En esta cadena, la entrada del usuario se pasa a la plantilla de prompt, luego la salida de la plantilla de prompt se pasa al modelo, y finalmente, la salida del modelo se pasa al analizador de salida. Echemos un vistazo a cada componente individualmente para entender realmente lo que está sucediendo.

## 1.  Prompt
`prompt` es un `BasePromptTemplate`, lo que significa que toma un diccionario de variables de plantilla y produce un `PromptValue`. Un `PromptValue` es un contenedor alrededor de un prompt completado que se puede pasar tanto a un LLM (que toma una cadena como entrada) como a un ChatModel (que toma una secuencia de mensajes como entrada). Puede trabajar con ambos tipos de modelos de lenguaje porque define lógica tanto para producir `BaseMessages` como para producir una cadena.